# Training neural network to predict ball position in a 3d space
## 1. Importing Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from utils.evaluate import evaluate_model
from utils.plot_maker_gaussian_process import plot_maker_3d

## 2. Importing Data

In [2]:
# get all train data
X_butterfly_a = pd.read_csv('../data-4cams/data-butterfly/red_ball_coordinates_128px_2024-03-08_14h11m.csv', decimal='.', delimiter=',')
X_butterfly_b = pd.read_csv('../data-4cams/data-butterfly/red_ball_coordinates_128px_2024-03-08_14h19m.csv', decimal='.', delimiter=',')
X_butterfly_c = pd.read_csv('../data-4cams/data-butterfly/red_ball_coordinates_128px_2024-03-08_14h23m.csv', decimal='.', delimiter=',')
X_spiral_a = pd.read_csv('../data-4cams/data-spiral/red_ball_coordinates_128px_2024-03-08_14h30m.csv', decimal='.', delimiter=',')[:700]
X_spiral_b = pd.read_csv('../data-4cams/data-spiral/red_ball_coordinates_128px_2024-03-08_14h34m.csv', decimal='.', delimiter=',')
X_spiro_a = pd.read_csv('../data-4cams/data-spiro/red_ball_coordinates_128px_2024-03-08_14h38m.csv', decimal='.', delimiter=',')
X_spiro_b = pd.read_csv('../data-4cams/data-spiro/red_ball_coordinates_128px_2024-03-08_14h41m.csv', decimal='.', delimiter=',').iloc[:800]
X_lis_a = pd.read_csv('../data-4cams/data-lis-curve/red_ball_coordinates_128px_2024-03-12_11h08m.csv', decimal='.', delimiter=',')
X_lis_b = pd.read_csv('../data-4cams/data-lis-curve/red_ball_coordinates_128px_2024-03-12_12h50m.csv', decimal='.', delimiter=',')

X_circle_a = pd.read_csv('../data-4cams/data-circle/red_ball_coordinates_128px_2024-03-12_12h59m.csv',decimal='.', delimiter=',')
X_circle_b = pd.read_csv('../data-4cams/data-circle/red_ball_coordinates_128px_2024-03-12_13h04m.csv',decimal='.', delimiter=',')

In [3]:
# get all the position data
y_butterfly_a = pd.read_csv('../data-4cams/data-butterfly/positions_xyz_2024-03-08_14h11m.csv', decimal='.', delimiter=',').iloc[4:]
y_butterfly_b = pd.read_csv('../data-4cams/data-butterfly/positions_xyz_2024-03-08_14h19m.csv', decimal='.', delimiter=',').iloc[4:]
y_butterfly_c = pd.read_csv('../data-4cams/data-butterfly/positions_xyz_2024-03-08_14h23m.csv', decimal='.', delimiter=',').iloc[4:]
y_spiral_a = pd.read_csv('../data-4cams/data-spiral/positions_xyz_2024-03-08_14h30m.csv', decimal='.', delimiter=',').iloc[4:][:700]
y_spiral_b = pd.read_csv('../data-4cams/data-spiral/positions_xyz_2024-03-08_14h34m.csv', decimal='.', delimiter=',').iloc[4:]
y_spiro_a = pd.read_csv('../data-4cams/data-spiro/positions_xyz_2024-03-08_14h38m.csv', decimal='.', delimiter=',').iloc[4:]
y_spiro_b = pd.read_csv('../data-4cams/data-spiro/positions_xyz_2024-03-08_14h41m.csv', decimal='.', delimiter=',').iloc[4:]
y_spiro_b = y_spiro_b.iloc[:800]
y_lis_a = pd.read_csv('../data-4cams/data-lis-curve/positions_xyz_2024-03-12_11h08m.csv', decimal='.', delimiter=',').iloc[4:]
y_lis_b = pd.read_csv('../data-4cams/data-lis-curve/positions_xyz_2024-03-12_12h50m.csv', decimal='.', delimiter=',').iloc[4:]
y_circle_a = pd.read_csv('../data-4cams/data-circle/positions_xyz_2024-03-12_12h59m.csv', decimal='.', delimiter=',').iloc[4:]
y_circle_b = pd.read_csv('../data-4cams/data-circle/positions_xyz_2024-03-12_13h04m.csv', decimal='.', delimiter=',').iloc[4:]

#### 2.1 Check for Missing Values in spriral_down_b and spiral_down_c

In [4]:
print(f'The amount of NaN in spiral_down_b: {X_butterfly_a.isna().sum().sum()}')
print(f'The amount of NaN in spiral_down_c: {X_butterfly_b.isna().sum().sum()}')
print(f'The amount of NaN in butterfly_a: {X_butterfly_c.isna().sum().sum()}')
print(f'The amount of NaN in spiral_a: {X_spiral_a.isna().sum().sum()}')
print(f'The amount of NaN in spiral_b: {X_spiral_b.isna().sum().sum()}')
print(f'The amount of NaN in spiro_a: {X_spiro_a.isna().sum().sum()}')
print(f'The amount of NaN in spiro_b: {X_spiro_b.isna().sum().sum()}')
print(f'The amount of NaN in lis_a: {X_lis_a.isna().sum().sum()}')
print(f'The amount of NaN in lis_b: {X_lis_b.isna().sum().sum()}')
print(f'The amount of NaN in circle_a: {X_circle_a.isna().sum().sum()}')
print(f'The amount of NaN in circle_b: {X_circle_b.isna().sum().sum()}')

The amount of NaN in spiral_down_b: 6
The amount of NaN in spiral_down_c: 2
The amount of NaN in butterfly_a: 6
The amount of NaN in spiral_a: 0
The amount of NaN in spiral_b: 8
The amount of NaN in spiro_a: 0
The amount of NaN in spiro_b: 4
The amount of NaN in lis_a: 0
The amount of NaN in lis_b: 0
The amount of NaN in circle_a: 0
The amount of NaN in circle_b: 0


In [5]:
X_train = pd.concat([X_butterfly_a, X_butterfly_b, X_butterfly_c, X_spiral_a, X_spiral_b, X_spiro_a, X_spiro_b, X_lis_a, X_lis_b, X_circle_a, X_circle_b])
y_train = pd.concat([y_butterfly_a, y_butterfly_b, y_butterfly_c, y_spiral_a, y_spiral_b, y_spiro_a, y_spiro_b, y_lis_a, y_lis_b, y_circle_a, y_circle_b])
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_train.shape, y_train.shape

((11543, 8), (11543, 3))

In [6]:
combined = pd.concat([X_train, y_train], axis=1)
combined.isna().sum().sum()
combined.dropna(inplace=True)

In [7]:
print(f'The amount of NaN in the combined data: {combined.isna().sum().sum()}')

The amount of NaN in the combined data: 0


## 3. Data Preprocessing

In [8]:
X = combined.iloc[:, :8]
y = combined.iloc[:, 8:]
X.to_csv('../data-4cams/all/red_ball_all_4cams.csv', index=False)
y.to_csv('../data-4cams/all/positions_xyz_all_4cams.csv', index=False)

In [9]:
# scale the data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

## 4. Hyperparameter Tuning

In [10]:
# create a pipeline
pipeline = Pipeline([
    ('mlp', MLPRegressor(random_state=42))
])

In [ ]:
param_grid = {
    'mlp__hidden_layer_sizes': [(10,), (50, 30), (100, 50), (20, 10, 5)],
    'mlp__activation': ['identity', 'tanh', 'relu'],
    'mlp__solver': ['sgd', 'adam'],
    'mlp__alpha': [0.001, 0.01, 0.1],
    'mlp__max_iter': [10000, 20000, 28000],
    'mlp__learning_rate': ['constant', 'adaptive'],
    'mlp__learning_rate_init': [0.001, 0.01, 0.1],
    'mlp__early_stopping': [True],
}

random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, n_iter=100, n_jobs=-1, verbose=3, random_state=42)

random_search.fit(X_scaled, y_scaled)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
print(f'Best parameters: {random_search.best_params_}')
print(f'Best score: {random_search.best_score_}')

In [ ]:
model = random_search.best_estimator_

## 5. Training the Model

In [ ]:
model.fit(X_scaled, y_scaled)

## 6. Make Predictions On Test Data

In [ ]:
# get all test data
X_test_8 = pd.read_csv('../data-4cams/data-8/red_ball_coordinates_128px_2024-03-08_14h44m.csv', decimal='.', delimiter=',')
y_test_8 = pd.read_csv('../data-4cams/data-8/positions_xyz_2024-03-08_14h44m.csv', decimal='.', delimiter=',').iloc[4:]

In [ ]:
X_lis_c = pd.read_csv('../data-4cams/data-lis-curve/red_ball_coordinates_128px_2024-03-12_12h54m.csv', decimal='.', delimiter=',')
y_lis_c = pd.read_csv('../data-4cams/data-lis-curve/positions_xyz_2024-03-12_12h54m.csv', decimal='.', delimiter=',').iloc[4:]

In [ ]:
def prepare_and_scale_test_data(X_test_val, y_test_val, scaler_x_test):
    y_test_val = y_test_val.iloc[4:]
    combined_test = pd.concat([X_test_val, y_test_val], axis=1)
    combined_test.dropna(inplace=True)
    X_val = combined_test.iloc[:, :8]
    y_val = combined_test.iloc[:, 8:]
    X_scaled_test = scaler_x_test.transform(X_val)
    return X_scaled_test, y_val

In [ ]:
def predict_evaluate_plot(X_test_val_scaled, y_test_val, nn_model, scaler_y_test, title='Trajectory of Ball in 3D Space'):
    y_test_predicted_scaled = nn_model.predict(X_test_val_scaled)
    y_test_pred = scaler_y_test.inverse_transform(y_test_predicted_scaled)
    mse, mae, r2 = evaluate_model(y_test_val, y_test_pred, 'Test Data')
    # change col 1 and 2 from place, so it corresponds to the unity 3d space
    y_test_pred = y_test_pred[:, [0, 2, 1]]
    y_test_val = y_test_val.to_numpy()[:, [0, 2, 1]]
    plot_maker_3d(y_test_val, y_test_pred, title)
    return y_test_pred, mse, mae, r2

In [ ]:
X_test_scaled_8, y_test_8 = prepare_and_scale_test_data(X_test_8, y_test_8, scaler_x)
y_test_pred_8, mse_8, mae_8, r2_8 = predict_evaluate_plot(X_test_scaled_8, y_test_8, model, scaler_y, 'Trajectory of Ball in 3D Space - 8 motion')

In [ ]:
X_test_scaled_lis_c, y_test_lis_c = prepare_and_scale_test_data(X_lis_c, y_lis_c, scaler_x)
y_test_pred_lis_c, mse_lis_c, mae_lis_c, r2_lis_c = predict_evaluate_plot(X_test_scaled_lis_c, y_test_lis_c, model, scaler_y, 'Trajectory of Ball in 3D Space - Lissajous Curve')
